# LCEL Interface


## Overview

The LangChain Expression Language (LCEL) is a 強大的介面, 用於簡化在 LangChain 中建立和管理自定義鏈的流程. 
LCEL 實現了 Runnable 協議, 提供了一種標準化的方式來構建和執行語言模型鏈.


### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [LCEL Runnable Protocol](#lcel-runnable-protocol)
- [stream: real-time output](#stream-real-time-output)
- [Invoke](#invoke)
- [batch: unit execution](#batch-unit-execution)
- [async stream](#async-stream)
- [async invoke](#async-invoke)
- [async batch](#async-batch)
- [Parallel](#parallel)
- [Parallelism in batches](#parallelism-in-batches)

### References

- [Langsmith DOC](https://docs.smith.langchain.com/)
---

## Environment Setup

設定環境. 你可以到 [Environment Setup](https://wikidocs.net/257836) 了解更多詳細信息.

**[Note]**
- ```langchain-opentutorial``` 是一個提供一套簡單易用的環境設定, 有用的功能和工具的套件.
- 你可以到 [```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) 了解更多詳細信息.

set environment variables is in .env.

Copy the contents of .env_sample and load it into your .env with the key you set.

In [1]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [9]:
%%capture --no-stderr
!pip install langchain-opentutorial


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain-openai",
        "langchain",
        "python-dotenv",
        "langchain-core",
    ],
    verbose=False,
    upgrade=False,
)

In [11]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "<Your OpenAI API KEY>",
        "LANGCHAIN_API_KEY": "<Your LangChain API KEY>",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "LangSmith-Tracking-Setup",  # title 과 동일하게 설정해 주세요
    }
)

Environment variables have been set successfully.


## LCEL Runnable Protocol

---

為了讓建立自定義鏈的流程盡可能簡單, 我們實現了 ```Runnable``` 協議.

```Runnable``` 協議在大多數組件中都已實現.

```Runnable``` 協議提供了一個標準介面, 使定義自定義鏈和以標準方式調用它們變得容易. 標準介面包括

- ```stream```: 流式輸出一個響應的片段.
- ```invoke```: 在輸入上調用鏈.
- ```batch```: 在輸入列表上調用鏈.

還有異步方法

- ```astream```: 異步流式輸出一個響應的片段.
- ```ainvoke```: 在輸入上異步調用鏈.
- ```abatch```: 在輸入列表上異步調用鏈.
- ```astream_log```: 異步流式輸出最終響應以及在發生時的中間步驟.



### Log your trace

我們提供多種方式將追蹤記錄到 LangSmith. 下面, 我們將突出顯示如何使用 traceable().

使用以下代碼在 LangSmith 中記錄追蹤

In [ ]:
import openai
from langsmith import wrappers, traceable

# 自動追蹤 LLM 調用
client = wrappers.wrap_openai(openai.Client())

@traceable # 自動追蹤此函數
def pipeline(user_input: str):
    result = client.chat.completions.create(
        messages=[{"role": "user", "content": user_input}],
        model="gpt-4o-mini"
    )
    return result.choices[0].message.content

pipeline("Hello, world!")
# Out:  Hello there! How can I assist you today?

'Hello! How can I assist you today?'

Create a chain using LCEL syntax.

In [15]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Instantiate the ChatOpenAI model.
model = ChatOpenAI()
# Create a prompt template that asks for jokes on a given topic.
prompt = PromptTemplate.from_template("Describe the {topic} in 3 sentences.")
# Connect the prompt and model to create a conversation chain.
chain = prompt | model | StrOutputParser()

## stream: real-time output

這個 function 使用 ```chain.stream``` method 來創建一個給定主題的數據流, 並迭代輸出每個數據片段的 ```content```. 
```end=""``` argument 禁用輸出後的換行, ```flush=True``` argument 強制立即清空輸出緩衝區.

In [ ]:
# 使用 chain.stream method 創建一個給定主題的數據流, 並迭代輸出每個數據片段的 content. 
for token in chain.stream({"topic": "multimodal"}):
    # Output the content of each piece of data without newlines.
    print(token, end="", flush=True)

# example output 
# The multimodal approach involves using multiple modes of communication, such as visual, auditory, and kinesthetic, to enhance learning and understanding. By incorporating different sensory inputs, learners are able to engage with material in a more holistic and immersive way. This approach is especially effective in catering to diverse learning styles and preferences.

The multimodal approach involves using multiple modes of communication, such as visual, auditory, and kinesthetic, to enhance learning and understanding. By incorporating various modalities, individuals can access information in ways that cater to their unique learning styles and preferences. This approach allows for a more comprehensive and inclusive learning experience that promotes deeper engagement and retention of information.

## Invoke

The ```invoke``` method of a ```chain``` object takes a topic as an argument and performs processing on that topic.






In [17]:
# Call the invoke method of the chain object, passing a dictionary with the topic 'ChatGPT'.
chain.invoke({"topic": "ChatGPT"})

'ChatGPT is a state-of-the-art conversational AI model developed by OpenAI that can engage in natural and coherent conversations with users. It uses the GPT-3 architecture, which allows it to generate human-like responses based on the input it receives. ChatGPT is known for its ability to understand context, provide relevant information, and maintain engaging conversations across a wide range of topics.'

## batch: unit execution

The function ```chain.batch``` takes a list containing multiple dictionaries as arguments and performs batch processing using the values of the ```topic``` key in each dictionary.

In [18]:
# Call a function to batch process a given list of topics
chain.batch([{"topic": "ChatGPT"}, {"topic": "Instagram"}])

['ChatGPT is an advanced conversational AI model developed by OpenAI that can engage in natural and engaging conversations with users. It is trained on a diverse range of internet text data to understand and generate human-like responses. ChatGPT is capable of providing information, answering questions, and engaging in meaningful dialogues with users on a variety of topics.',
 'Instagram is a social media platform where users can share photos and videos with their followers. The app allows users to edit and enhance their photos with filters and captions before posting. Users can also interact with others by liking, commenting, and messaging on their posts.']

你可以使用 ```max_concurrency``` parameter 來設定同時請求的數量.

```config``` dictionary 使用 ```max_concurrency``` key 來設定可以同時處理的最大操作數量. 在這裡, 它被設定為同時處理最多三個工作.

In [19]:
chain.batch(
    [
        {"topic": "ChatGPT"},
        {"topic": "Instagram"},
        {"topic": "multimodal"},
        {"topic": "programming"},
        {"topic": "machineLearning"},
    ],
    config={"max_concurrency": 3},
)

['ChatGPT is an AI-powered chatbot developed by OpenAI that can engage in text-based conversations with users. It uses a machine learning model trained on a diverse range of internet text to generate responses that are contextually relevant and engaging. ChatGPT can provide information, answer questions, or just have a casual conversation with users.',
 'Instagram is a social media platform where users can share photos and videos with their followers. It allows users to edit and enhance their photos with filters and editing tools. Users can also engage with others by liking, commenting, and sharing posts.',
 'The multimodal approach involves using multiple modes of communication, such as visual, auditory, and kinesthetic, to enhance learning and understanding. This approach recognizes that individuals learn in different ways and can benefit from a variety of sensory experiences. By incorporating different modes of communication, educators can cater to diverse learning styles and create

## async stream

The 函數 ```chain.astream``` 建立一個異步流，並異步處理給定主題的訊息。

它使用異步 for 迴圈 (```async for```) 來序列接收流中的訊息，並使用 print 函數立即印出訊息的內容 (```s.content```)。 ```end=""``` 禁用印出後的行包裝，而 ```flush=True``` 強制輸出緩衝區被清空以確保立即印出。


In [ ]:
# 使用異步流來處理 'YouTube' 主題的訊息。
async for token in chain.astream({"topic": "YouTube"}):
    # 印出訊息內容。直接輸出，不換行，並清空緩衝區。
    print(token, end="", flush=True)

YouTube is a video-sharing platform where users can upload, view, and interact with videos on a wide range of topics. It has become one of the largest and most popular websites on the internet, with millions of users visiting daily. Users can subscribe to channels, like and comment on videos, and create playlists to save their favorite content.

## 非同步調用
ainvoke 是 chain 物件的一個方法，用於以非同步方式執行操作。在此例中，我們傳入一個字典作為參數，該字典包含一個鍵為 topic 且值為 NVDA（NVIDIA 的股票代碼）的項目。此方法可用於非同步請求處理特定主題。

這個翻譯保持了技術術語的準確性，同時確保了中文表達的自然流暢。

In [ ]:
# 處理 'NVDA' 主題的訊息。
my_process = chain.ainvoke({"topic": "NVDA"})

In [ ]:
# 等待非同步處理完成。
await my_process

'The National Visa Center (NVDA) is a government agency responsible for processing immigrant visa applications for individuals seeking to live permanently in the United States. They review and approve the necessary documentation, schedule interviews at U.S. embassies or consulates, and ensure that applicants meet all eligibility requirements for a visa. The NVDA plays a crucial role in managing the visa application process and facilitating legal immigration to the United States.'

# 非同步批次處理

函數 ```abatch``` 以非同步方式批次處理一系列操作。

在這個例子中，我們使用 ```chain``` 物件的 ```abatch``` 方法以非同步方式處理 ```topic``` 的操作。

```await``` 關鍵字用於等待這些非同步任務完成。

In [ ]:
# 非同步批次處理
my_abatch_process = chain.abatch(
    [{"topic": "YouTube"}, {"topic": "Instagram"}, {"topic": "Facebook"}]
)

In [ ]:
# 等待非同步批次處理完成。
await my_abatch_process

['YouTube is a popular video-sharing platform where users can upload, view, and share videos on a wide range of topics. It has become a major source of entertainment, education, and information for millions of people around the world. With features like live streaming, monetization options, and a vast library of content, YouTube has revolutionized the way we consume video content.',
 'Instagram is a popular social media platform where users can share photos and videos with their followers. It allows for creative expression through filters, captions, and hashtags. Users can also interact with others by liking, commenting, and messaging on posts.',
 'Facebook is a popular social media platform where users can connect with friends and family, share photos and updates, and join groups based on their interests. It was founded by Mark Zuckerberg in 2004 and has since grown to have billions of active users worldwide. Users can also follow pages and businesses to stay updated on news and event

# 平行處理 Parallel

讓我們看看 LangChain Expression Language 如何支持平行請求。例如，當您使用 ```RunnableParallel```（通常以字典形式寫入）時，您會平行執行每個元素。

以下是一個使用 ```langchain_core.runnables``` 模組中的 ```RunnableParallel``` 類別平行執行兩個任務的示例。

創建兩個鏈（```chain1```, ```chain2```)，使用 ```ChatPromptTemplate.from_template``` 方法獲取給定 ```country``` 的首都和面積。

這些鏈通過 ```model``` 和管道（```|```）操作符相連。最後，我們使用 ```RunnableParallel``` 類別將這兩個鏈與鍵 ```capital``` 和 ```area``` 連接起來，創建一個 ```combined``` 物件，該物件可以平行運行。

In [ ]:
from langchain_core.runnables import RunnableParallel

# Create a chain that asks for the capital of {country}.
chain1 = (
    PromptTemplate.from_template("What is the capital of {country}?")
    | model
    | StrOutputParser()
)

# Create a chain that asks for the area of {country}.
chain2 = (
    PromptTemplate.from_template("What is the area of {country}?")
    | model
    | StrOutputParser()
)

# Create a parallel execution chain that generates the above two chains in parallel.
combined = RunnableParallel(capital=chain1, area=chain2)

函數 ```chain1.invoke()``` 呼叫 ```chain1``` 物件的 ```invoke``` 方法。

作為參數，它傳遞一個字典，其中包含鍵名為 ```country``` 的值 ```Canada```。

In [26]:
# Run chain1 .
chain1.invoke({"country": "Canada"})

'Ottawa'

呼叫 ```chain2.invoke()```, 這次傳遞一個不同的國家，即 ```United States```, 作為 country 的鍵值。

In [27]:
# Run chain2 .
chain2.invoke({"country": "USA"})

'The total land area of the United States is approximately 3.8 million square miles (9.8 million square kilometers).'

函數 ```combined.invoke()``` 執行給定 ```country``` 的處理。

在這個例子中，主題 ```USA``` 作為 ```country``` 的鍵值傳遞給 ```invoke``` 方法以執行。

In [28]:
# Run a parallel execution chain.
combined.invoke({"country": "USA"})

{'capital': 'The capital of the United States of America is Washington, D.C.',
 'area': 'The total area of the United States is approximately 9.8 million square kilometers (3.8 million square miles).'}

# 平行處理在批次中

平行處理可以與其他可執行代碼結合使用。讓我們嘗試使用平行處理進行批次處理。

函數 ```chain1.batch``` 接受一個包含多個字典的列表作為參數，並處理每個字典中 "topic" 鍵對應的值。在這個例子中，我們正在批次處理兩個主題，"Canada" 和 "United States"。

In [29]:
# Perform batch processing.
chain1.batch([{"country": "Canada"}, {"country": "USA"}])


['Ottawa', 'The capital of the United States of America is Washington, D.C.']

函數 ```chain2.batch``` 接受多個字典的列表作為參數，並執行批次處理。

在這個例子中，我們請求處理兩個國家，```Canada``` 和 ```United States```。

In [30]:
# Perform batch processing.
chain2.batch([{"country": "Canada"}, {"country": "USA"}])


['The total land area of Canada is approximately 9.98 million square kilometers (3.85 million square miles), making it the second largest country in the world by land area.',
 'The total land area of the United States is approximately 3.8 million square miles.']

函數 ```combined.batch``` 用於將給定數據按批次處理。

在這個例子中，它接受一個包含兩個字典物件的列表作為參數，並分別對加拿大和美國的數據進行批次處理。

In [31]:
# Processes the given data in batches.
combined.batch([{"country": "Canada"}, {"country": "USA"}])


[{'capital': 'Ottawa',
  'area': 'The total area of Canada is approximately 9.98 million square kilometers (3.85 million square miles), making it the second-largest country in the world by land area.'},
 {'capital': 'The capital of the United States of America is Washington, D.C.',
  'area': 'The total land area of the United States is approximately 3.8 million square miles (9.8 million square kilometers).'}]